# Librerias

In [ ]:
import pandas as pd

# Dataset

## Lectura del dataset

In [ ]:
df0 = pd.read_parquet("train0.parquet")
df1 = pd.read_parquet("train1.parquet")
df2 = pd.read_parquet("train2.parquet")

In [ ]:
df = pd.concat([df0, df1, df2], axis=0)

## Sample del dataset

In [ ]:
df.shape

In [ ]:
# Se obtiene una muestra del dataset
df_train = df.sample(250000,random_state=42)
df_train.reset_index(drop=True, inplace=True)
df_train.head(5)

In [ ]:
df_train['text'] = df_train['review_headline'] + ' ' + df_train['review_body']

# Modelamiento por Arquitecturas

## Naives Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay

import re
import nltk
from nltk.corpus import stopwords
stemmer = nltk.stem.SnowballStemmer('english') 
nltk.download('stopwords') 

In [ ]:
def processing_text(texto):
    # Paso 1: Remover con un expresión regular carateres especiales (no palabras).
    processed_feature = re.sub(r'\W', ' ', str(texto))
    # Paso 2: Remover ocurrencias de caracteres individuales
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)
    # Paso 3: Remover números (Ocurrencias muy esporádicas en nuestro dataset)
    processed_feature = re.sub(r'[0-9]+', ' ', processed_feature)
    # Paso 4: Simplificar espacios concecutivos a un único espacio entre palabras
    processed_feature = re.sub(' +', ' ', processed_feature)
    # Paso 5: Pasar todo el texto a minúsculas
    processed_feature = processed_feature.lower()
    # Paso 6: Aplicar stemming. Es una forma de enviar las palabras a una raiz común simplificando de esta manera el vocabulario.
    #         por ejemplo las palabras (absurdo, absurdos) que estan en el review 2895 seran llevados a la raiz común "absurd"
    #         y de esta forma se evita tener dos palabras diferentes con el mismo significado en nuestro vocabulario.
    processed_feature = " ".join([stemmer.stem(i) for i in processed_feature.split()])
    return processed_feature

In [ ]:
# texto_para_procesar y labels respectivamente
texto_para_procesar = df_train['text'].values
labels = df_train['star_rating'].values

# El texto ya procesado de cada ejemplo en nuestro dataset lo almacenaremos en la variable "texto_procesado"
texto_procesado = []
for sentence in range(0, len(texto_para_procesar)):
    procesado = processing_text(texto_para_procesar[sentence])
    texto_procesado.append(procesado)

### Representacion 1

In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=2500, stop_words=stopwords.words('english'))
# max_features representa el tamaño del vocabulario. Vamos a permitir 2500 palabras.
# stop_words le indicamos las palabras de parada para que las ignore en el vocabulario.

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(texto_features, labels, test_size=0.2, random_state=0)

In [ ]:
# Ahora entrenemos un modelo simple. Ya conocenos Naive Bayes!!
nb = MultinomialNB()
# El método fit en sklearn permite ejecutar el proceso de entrenamiento.
nb.fit(X_train, y_train)

In [ ]:
predictions = nb.predict(X_test)

In [ ]:
# Ahora calculamos el score de accuracy enviando las predicciónes y los valores reales de polaridad.
print(accuracy_score(y_test, predictions))

In [ ]:
## OPCIONAL
## Vamos a profundizar en estos conceptos mas adelante.
cm = confusion_matrix(y_test, predictions,labels=nb.classes_)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=nb.classes_)
disp.plot()

In [ ]:
print(classification_report(y_test, predictions, digits=4))

### Representacion 2

### Representacion 3

## Logistic Regresion (SDGClassfier)

In [ ]:
from sklearn.linear_model import SGDClassifier

### Representacion 1

### Representacion 2


### Representacion 3

## Neuronal Network (Sequencial)

In [ ]:
from keras.models import Sequential

### Representacion 1

### Representacion 2

### Representacion 3